In [36]:
import sqlite3
from pathlib import Path
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import mlflow
from dotenv import load_dotenv
import os
import ast
import re

In [ ]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")


In [38]:
caminho_db = Path.cwd() / "data" / "extraidos" / "dados.db"
print(caminho_db)

d:\0Viviana\1 FIAP\5 fase\datathon\app\data\extraidos\dados.db


In [34]:
def buscar_dados(nome_candidato):
    conn = sqlite3.connect(caminho_db)
    cursor = conn.cursor()

    # Buscar candidato
    cursor.execute("""
        SELECT informacoes_pessoais_nome,
               informacoes_profissionais_area_atuacao,
               informacoes_profissionais_conhecimentos_tecnicos,
               informacoes_profissionais_certificacoes,
               informacoes_profissionais_nivel_profissional,
               informacoes_profissionais_experiencias,
               formacao_e_idiomas_nivel_academico,
               formacao_e_idiomas_nivel_ingles,
               formacao_e_idiomas_nivel_espanhol,
               infos_basicas_objetivo_profissional,
               cargo_atual_projeto_atual,
               cv_pt,
               infos_basicas_codigo_profissional  
        FROM applicants
        WHERE informacoes_pessoais_nome = ?
    """, (nome_candidato,))
    candidato = cursor.fetchone()

    if not candidato:
        conn.close()
        return None, None

    codigo_vaga = candidato[-1]

    # Buscar vaga
    cursor.execute("""
        SELECT informacoes_basicas_titulo_vaga,
               informacoes_basicas_objetivo_vaga,
               perfil_vaga_competencia_tecnicas_e_comportamentais,
               perfil_vaga_habilidades_comportamentais_necessarias
        FROM vagas
        WHERE codigo = ?
    """, (codigo_vaga,))
    vaga = cursor.fetchone()

    conn.close()
    return candidato, vaga, codigo_vaga

In [35]:
teste = buscar_dados("Carolina Aparecida")
teste

(('Carolina Aparecida',
  '',
  '',
  '',
  '',
  None,
  '',
  '',
  '',
  '',
  None,
  "assistente administrativo\n\n\nsantosbatista\nitapecerica da serra/sp\n29 anos ▪ brasileiro ▪ casado\nformação acadêmica\n\uf0b7 bacharel - ciências contábeis\ncentro universitário ítalo brasileiro\njul/2015 - dez/2018\n\uf0b7 graduação - gestão financeira\ncentro universitário anhanguera\njan/2013 - dez/2014\nhabilidades\n\uf0b7 contas a pagar e receber\n\uf0b7 excel avançado\n\uf0b7 indicadores kpi’s\n\uf0b7 notas fiscais, cfop’s\n\uf0b7 fechamento contábil\n\uf0b7 emissão de boletos\n\uf0b7 guias\n\uf0b7 impostos\n\uf0b7 budget\n\uf0b7 controladoria\n\uf0b7 sistemas integrados:\ntotvs;\nfolha matic;\nnavision\nresumo profissional\nprofissional com experiência nos departamentos financeiro,\ncontábil, fiscal e controladoria jurídica. elaboração e análise de\nindicadores kpi’s de resultado, relatórios, guias, gestão de\npagamentos, notas fiscais, boletos, fechamento financeiro e\ncontábil fiscal.

In [ ]:
teste = buscar_dados("Carolina Aparecida")
teste

In [25]:
# 🧠 Gerar perguntas com LangChain
def gerar_perguntas(candidato, vaga):
    llm = ChatOpenAI(temperature=0.7)

    nome, area, tecnicos, certificacoes, nivel_prof, nivel_acad, ingles, espanhol, _ = candidato
    titulo_vaga, objetivo_vaga, competencias_tecnicas, competencias_comportamentais = vaga

    resumo = f"""
    Nome: {nome}
    Área de atuação: {area}
    Conhecimentos técnicos: {tecnicos}
    Certificações: {certificacoes}
    Nível profissional: {nivel_prof}
    Nível acadêmico: {nivel_acad}
    Inglês: {ingles}, Espanhol: {espanhol}
    """

    competencias = f"{competencias_tecnicas}, {competencias_comportamentais}"

    prompt = ChatPromptTemplate.from_template("""
    Você é um agente de RH entrevistando um candidato para a vaga de "{titulo_vaga}".
    Objetivo da vaga: {objetivo_vaga}
    Perfil do candidato: {resumo}
    Competências exigidas: {competencias}

    Gere 5 perguntas para avaliar se o candidato possui as competências necessárias.
    Seja direto e profissional.
    """)

    chain = LLMChain(llm=llm, prompt=prompt)

    resposta = chain.run({
        "titulo_vaga": titulo_vaga,
        "objetivo_vaga": objetivo_vaga,
        "resumo": resumo,
        "competencias": competencias
    })

    # Log no MLflow
    mlflow.log_param("candidato", nome)
    mlflow.log_param("vaga", titulo_vaga)
    mlflow.log_param("competencias", competencias)
    mlflow.log_text(resumo, "resumo_candidato.txt")
    mlflow.log_text(resposta, "perguntas_geradas.txt")

    return resposta



In [26]:
# 🚀 Executar entrevista
def entrevista(nome_candidato):
    with mlflow.start_run(run_name=f"Entrevista_{nome_candidato}"):
        candidato, vaga = buscar_dados(nome_candidato)
        if not candidato or not vaga:
            print("Candidato ou vaga não encontrados.")
            return

        perguntas = gerar_perguntas(candidato, vaga)
        print(f"\n📋 Perguntas para entrevista com {nome_candidato}:\n")
        print(perguntas)

In [27]:
# 🧪 Exemplo de uso
entrevista("Carolina Aparecida")


Candidato ou vaga não encontrados.
